In [1]:
from obspy.core import Stream, read
import numpy as np
from pytensor.compile.ops import as_op
import pytensor
import pytensor.tensor as pt
import matplotlib.pyplot as plt
import pandas as pd
import os 
import os.path
import pymc as pm
import arviz as az

In [2]:
# What exactly are the colunns here? Amplitude of waves at a specific time?
# My equivalent = peak strain at a specific time

# Build the target function, misfit to this is what is being minimized
@as_op(itypes=[pt.dvector,pt.dscalar,pt.dscalar,pt.dscalar,pt.dscalar,pt.dscalar], otypes=[pt.dvector])
def two_straight_lines(x,m1,m2,xinter,x0,y0):
    '''
    input x coordiantes are in x
    slopes are m1 and m2
    intercept of left hand line is b1 
    intersection of two lines is at xinter
    
    Note that y intercept of second straight line is dependent on b1 and xinter
      and defined entirely by them (so that the lines touch).
    '''
    
    # Output vector
    y_out = np.ones(len(x))
    
    # Before building the first straight line, calculate the intercept
    b1 = y0 - m1*x0
    
    # Build first straight line segment
    y_out = m1*x + b1
    
    # Find points that are after the intersection and make the second segment
    i = np.where(x>xinter)[0]
    
    # Define second y intercept
    b2 = m1*xinter + b1 - m2*xinter
    
    # Make second straight line
    y_out[i] = m2*x[i] + b2

    return y_out

# Build the target function, misfit to this is what is being minimized
def non_pytensor_two_straight_lines(x,m1,m2,xinter,x0,y0):
    '''
    input x coordiantes are in x
    slopes are m1 and m2
    intercept of left hand line is b1 
    intersection of two lines is at xinter
    
    Note that y intercept of second straight line is dependent on b1 and xinter
      and defined entirely by them (so that the lines touch).
    '''
    
    # Output vector
    yout = np.ones(len(x))
    
    # Before building the first straight line, calculate the intercept
    b1 = y0 - m1*x0
    
    # Build first straight line segment
    yout = m1*x + b1
    
    # Find points that are after the intersection and make the second segment
    i = np.where(x>xinter)[0]
    
    # Define second y intercept
    b2 = m1*xinter + b1 - m2*xinter
    
    # Make second straight line
    yout[i] = m2*x[i] + b2

    return yout

In [3]:
# Events

path_to_files = '/Users/sydneydybing/StrainProject/2024/'

earthquakes = pd.read_csv(path_to_files + 'M6_catalog_2004-2024.csv', dtype = str)

In [4]:
# earthquakes

In [5]:
weird_events_only = False
if weird_events_only:
    earthquakes = earthquakes.iloc[[4,15]]

small_test = False
if small_test:
    earthquakes = earthquakes.iloc[[11,12,13]]
    
one_at_a_time = False
if one_at_a_time:
#     earthquakes = earthquakes[:1]
    earthquakes = earthquakes.iloc[[29]]

origin_times = earthquakes.time.values
mags = earthquakes.mag.values

# Stations

bsm_mda = pd.read_csv(path_to_files + 'bsm_metadata.csv')
NA_only = bsm_mda.loc[bsm_mda['LONG'] < -110]
stas = NA_only['BNUM'].values

eqs_test = earthquakes[4:5]
ots_test = origin_times[4:5]
mags_test = mags[4:5]
rows_for_extra_times = [8,9,14,16]

dist_array = np.load(path_to_files + 'NA_evt-sta_dist_array_less500km_withdata_withrmssnrs_FILTERED.npy')

# Array order:

# 0. Earthquake identifier
# 1. Earthquake origin time
# 2. Earthquake latitude
# 3. Earthquake longitude
# 4. Earthquake magnitude
# 5. Station name
# 6. Station latitude
# 7. Station longitude
# 8. Hypocentral distance (km)
# 9. RMS strain SNR

In [7]:
# stas = ['B027', 'B028']

events_list = []
mags_list = []
stas_list = []
hypdist_list = []
xhighs_list = []
m1s_list = []
m2s_list = []
xinters_list = []
sigmas_list = []
trans_time_list = []

counter = 0

for idx in range(len(earthquakes)):
        
    ot = origin_times[idx]
    
    if idx in rows_for_extra_times:
        cut_ot = ot[:13]
    else:
        cut_ot = ot[:10]
    
    eq_id = str(cut_ot) + '_M' + str(mags[idx])
    print(eq_id)
    
    # Find rows that match this event in the array with the hypocentral distances
    
    u = np.where(dist_array[:,0] == eq_id)[0]
    # print(dist_array[u])
    
    for sta in stas:
        
        # print(sta)
        
        try:
            # Find row that matches this station within the rows of the right event
            j = np.where(dist_array[u,5] == str(sta))[0]
            hypdist = float(dist_array[u][j][0][8])
            int_hypdist = int(hypdist)
            snr = int(float(dist_array[u][j][0][9]))
            
            # Figure out the bounds for the MCMC based on the event and hypocentral distance (rainbow plots)
            
            if eq_id == '2023-04-13_M6':
                xhigh = 60
            elif eq_id == '2022-12-20_M6.4':
                if hypdist < 200:
                    xhigh = 20
                else:
                    xhigh = 40
            elif eq_id == '2022-11-22_M6.2': # Weird one that pauses for a while then grows again
                xhigh = 25
            elif eq_id == '2021-12-20_M6.2':
                if hypdst < 30: 
                    xhigh = 20
                elif 30 < hypdist < 50:
                    xhigh = 25
                elif 50 < hypdist < 100:
                    xhigh = 30
                else:
                    xhigh = 60
            elif eq_id == '2021-07-08_M6': # Lots of more gradual ones with pause again
                if hypdist < 300:
                    xhigh = 25
                else:
                    xhigh = 50
            elif eq_id == '2020-05-15_M6.5':
                xhigh = 30
            elif eq_id == '2020-03-31_M6.5':
                xhigh = 35
            elif eq_id == '2019-12-25_M6.3':
                xhigh = 35
            elif eq_id == '2019-12-23T20_M6':
                xhigh = 60
            elif eq_id == '2019-12-23T19_M6':
                xhigh = 60
            elif eq_id == '2019-08-29_M6.3': # All flatten fast
                xhigh = 60
            elif eq_id == '2019-07-06_M7.1': # Ridgecrest mainshock
                xhigh = 27
            elif eq_id == '2019-07-04_M6.4': # Ridgecrest foreshock
                if hypdist < 50:
                    xhigh = 20
                else:
                    xhigh = 27
            elif eq_id == '2019-07-04_M6.2':
                xhigh = 35
            # elif eq_id == '2018-10-22T06_M6.5': # No waveforms with high enough SNR
            #     xhigh = 
            elif eq_id == '2018-10-22_M6.8': # Has like 3 slopes? Letting it go long
                xhigh = 60
            elif eq_id == '2018-10-22T05_M6.5': # Somewhat of a pause but all similar? Going long
                xhigh = 60
            elif eq_id == '2018-08-22_M6.2':
                xhigh = 60
            elif eq_id == '2016-12-08_M6.6':
                xhigh = 60
            elif eq_id == '2014-08-24_M6.02':
                xhigh = 27
            elif eq_id == '2014-04-24_M6.5':
                xhigh = 50
            elif eq_id == '2014-03-10_M6.8':
                if hypdist < 110:
                    xhigh = 25
                else:
                    xhigh = 40
            elif eq_id == '2013-09-04_M6':
                xhigh = 25
            elif eq_id == '2013-09-03_M6.1':
                xhigh = 30
            elif eq_id == '2012-12-14_M6.3':
                xhigh = 25
            elif eq_id == '2012-11-08_M6.1':
                xhigh = 42
            elif eq_id == '2012-04-11_M6':
                xhigh = 30
            elif eq_id == '2011-09-09_M6.4':
                xhigh = 35
            elif eq_id == '2010-04-04_M7.2':
                xhigh = 35
            elif eq_id == '2010-01-10_M6.5':
                xhigh = 35
            elif eq_id == '2008-01-10_M6.3':
                xhigh = 24
            else:
                xhigh = 'no data'

            # Read in data - peak strain data only has the ones with SNR > 2
            pst = read(path_to_files + 'strain_data/filtered_peak_post_p/' + str(eq_id) + '/' + str(sta) + '.mseed')
            counter += 1
            print(counter)
            # pst.plot()
            # plt.plot(pst[0].times(), pst[0].data)
            # plt.semilogy()
            
            print('Processing ' + str(sta))

        except:
            continue # skips to next station
            
        times = pst[0].times()
        data = pst[0].data
        log10_data = np.log10(data)
        samp_rate = pst[0].stats.sampling_rate
        
#         print(times)
#         print(data)
        
#         plt.plot(times, data)

        xlow = 10
        
        # print('MCMC bounds:')
        # print(xlow, xhigh)
        
        vec_start = int(xlow*samp_rate)
        vec_end = int(xhigh*samp_rate)

        # Split into x and y vectors
        xobserved = times[vec_start:vec_end]
        # print(xobserved.shape)
        
        yobserved = log10_data[vec_start:vec_end]
        # print(yobserved.shape)
        
        # plt.plot(xobserved, yobserved)
        
        x0 = xobserved[0]
        y0 = yobserved[0]
        
        # in order to pass the x variable into the target function it needs to be 
        # converted to a Theano "shared" variable
        pt_xobserved = pytensor.shared(xobserved)
        pt_x0 = pytensor.shared(x0)
        pt_y0 = pytensor.shared(y0)
            
        # MCMC run parameters, these are good numbers for a "production" run. If you are
        # fooling arund these can be lower to iterate faster
        Nburn = 1000 # burn in samples that get discarded
        Nmcmc = 9000 # bump to at least 5-10k
        Nchains = 4
        Ncores = 1
        
        # Bounds for the prior distributions
        m1_low = 0 ; m1_high = 100 # lowest slope 0, highest 5
        m2_low = 0 ; m2_high = 10
        b1_low = -50 ; b1_high = 0 # lowest y-intercept -20, highest 0
        xinter_low = 10 ; xinter_high = 25 # location of the line slope change
        
        # Define the Bayesian model
        with pm.Model()as model:
            
            # Use normal distributions as priors
            m1 = pm.Normal('m1', mu=0.5, sigma=1)
            m2 = pm.Normal('m2', mu=-0.1,sigma=5)
            # b1 = pm.Normal('b1', mu=-5, sigma=5)
            xinter = pm.Uniform('xinter', lower = xinter_low, upper = xinter_high)
            sigma = pm.HalfCauchy('sigma', beta = 10, initval = 1.)
        
            # This is the model
            likelihood = pm.Normal('y', mu = two_straight_lines(pt_xobserved,m1,m2,xinter,pt_x0,pt_y0),
                                    observed = yobserved, sigma = sigma)
        #    likelihood = pm.Normal('y', mu=one_straight_line(xobserved,m1,b1),observed=yobserved,
        #                           sigma=sigma)
            
            # NUTS sampler (default) is gradient based and won't work, use metropolis
            step = pm.Metropolis()
            
            # This runs the mcmc sampler
            mcmc = pm.sample(Nmcmc, tune = Nburn, chains = Nchains, cores = Ncores, step = step)
        
        # Done, now is post-processing to get the data out of the sampler

        posterior = az.extract(mcmc)
        
        # Unwrap coeficients - WRITE LINES TO SAVE THE ARRAYS!!!
        m1_array = posterior.m1.values
        m1 = np.mean(m1_array)
        # print('m1 mean: ' + str(round(m1,2)))
        
        m2_array = posterior.m2.values
        m2 = np.mean(m2_array)
        # print('m2 mean: ' + str(round(m2,2)))
        
        xinter_array = posterior.xinter.values
        xinter = np.mean(xinter_array)
        # print('xinter Mean: ' + str(round(xinter,2)))
        
        sigma_array = posterior.sigma.values
        sigma = np.mean(sigma_array)

        # Lines!
        b1 = y0 - m1*x0
        b2 = m1*xinter + b1 - m2*xinter
        
        # Make plot to check stuff
        xpredicted = np.arange(xobserved.min(), xobserved.max()+0.1, 0.1)
        # ypredicted = np.ones(len(xpredicted))
        ypredicted = m1*xpredicted + b1
        i = np.where(xpredicted > xinter)[0]
        ypredicted[i] = m2*xpredicted[i] + b2
        
        # Get one-sigma region (need to obtain a ton of forward models and get stats)
        N = len(posterior.m1.values)
        
        yfit = np.zeros((len(xpredicted), N))
        for k in range(N):
            yfit[:,k] = non_pytensor_two_straight_lines(xpredicted,m1_array[k],m2_array[k],xinter_array[k],x0,y0)
        mu = yfit.mean(1)
        sig = yfit.std(1) * 1.95 #for 95% confidence
        mu_plus = mu + sig
        mu_minus = mu - sig
        
        # Least squares
        mls, bls = np.polyfit(xobserved, yobserved, 1)
        
        # Find the transition point in ypredicted
        diff = np.round(np.diff(ypredicted), decimals = 5)
        # print(diff)
        first_slope = diff[0]
        second_slope = diff[-1]
        trans_idx = np.where((diff != first_slope) & (diff != second_slope))[0][0] + 1
        
        # Get the transition point
        mcmc_trans_x = xpredicted[trans_idx] # Number
        mcmc_trans_y = ypredicted[trans_idx]
        
        # Append things
        events_list.append(eq_id)
        mags_list.append(mags[idx])
        stas_list.append(sta)
        hypdist_list.append(int_hypdist)
        xhighs_list.append(xhigh)
        m1s_list.append(m1)
        m2s_list.append(m2)
        xinters_list.append(xinter)
        sigmas_list.append(sigma)
        trans_time_list.append(mcmc_trans_x)

        # Plot all solutions
        fig = plt.figure(dpi = 100)
        ax = fig.add_subplot(111)
        ax.set_title(str(cut_ot) + '_M' + str(mags[idx]) + ': ' + str(sta))
        
        ax.plot(xobserved, yobserved, color = 'blue', label = 'Observed (log peak strain)')
        ax.plot(xpredicted, ypredicted, color = 'red', label = 'Predicted (MCMC)')
        # ax.plot(xpredicted, mu, color = 'green', linestyle = '--', label = 'yfit')
        # ax.plot(xpredicted, xpredicted * mls + bls, color = 'orange', label = 'np.polyfit least squares')
        ax.scatter(xpredicted[trans_idx], ypredicted[trans_idx], marker = 'o', color = 'black', label = 'Transition at ' + str(round(xpredicted[trans_idx],2)) + ' sec')
        # ax.scatter(closest_trans_x, closest_trans_y, marker = 'o', color = 'green', label = 'Adjusted transition at ' + str(round(closest_trans_x,2)) + ' sec')
        # ax2 = ax.twinx()
        # ax2.plot(xobserved, pythags_arr, color = 'violet')
        # ax2.axhline(closest_trans_x, color = 'violet', linestyle = '--')
        ax.fill_between(xpredicted, mu_plus, mu_minus, color = 'gray', alpha = 0.2, label = '95% confidence') # 95% confidence interval
        ax.set_xlabel('Time (s) - p-wave at 10s')
        ax.set_ylabel('log10 of peak strain')
        # ax.set_xlim(10,13)
        ax.legend()
        
        # plt.show()
        
        # Create folder for the event
        array_save_dir = path_to_files + 'filtered_mcmc_arrays/' + str(cut_ot) + '_M' + str(mags[idx]) + '/' + str(sta) + '/'
        if os.path.isdir(array_save_dir):
            pass
        else:
            os.makedirs(array_save_dir)

        # Save arrays
        np.save(array_save_dir + 'xobserved.npy', xobserved)
        np.save(array_save_dir + 'yobserved.npy', yobserved)
        np.save(array_save_dir + 'xpredicted.npy', xpredicted)
        np.save(array_save_dir + 'ypredicted.npy', ypredicted)
        np.save(array_save_dir + 'forward_mus.npy', mu)
        np.save(array_save_dir + 'foward_sigs.npy', sig)
        
        # Save posterior
        summary = az.summary(mcmc)
        summary.to_csv(array_save_dir + 'summary.csv')
        # posterior = az.extract(mcmc)
        # posterior.to_netcdf(path = array_save_dir + 'posterior.nc', mode = 'w') # Doesn't work right now
        
        # Save figure
        fig_save_dir = path_to_files + 'figures/filtered_mcmc_plots/' + str(cut_ot) + '_M' + str(mags[idx]) + '/'
        if os.path.isdir(fig_save_dir):
            pass
        else:
            os.makedirs(fig_save_dir)
            
        plt.savefig(fig_save_dir + sta + '.png', format = 'PNG', dpi = 400)
        plt.close()
        
        # print(str(sta) + ' done')

2023-04-13_M6
1
Processing B012


Sequential sampling (4 chains in 1 job)
CompoundStep
>Metropolis: [m1]
>Metropolis: [m2]
>Metropolis: [xinter]
>Metropolis: [sigma]


Sampling 4 chains for 1_000 tune and 9_000 draw iterations (4_000 + 36_000 draws total) took 59 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


2
Processing B927


Sequential sampling (4 chains in 1 job)
CompoundStep
>Metropolis: [m1]
>Metropolis: [m2]
>Metropolis: [xinter]
>Metropolis: [sigma]


KeyboardInterrupt: 

In [ ]:
mcmc_array = np.column_stack((np.array(events_list), np.array(mags_list), np.array(stas_list), np.array(hypdist_list),
                             np.array(xhighs_list), np.array(m1s_list), np.array(m2s_list), np.array(xinters_list),
                             np.array(sigmas_list), np.round(np.array(trans_time_list),2)))

np.save(path_to_files + 'filtered_mcmc_arrays/summary_array.npy', mcmc_array)

In [ ]:
mcmc_array.shape

In [ ]:
mcmc_array = np.load(path_to_files + 'filtered_mcmc_arrays/summary_array.npy')
mcmc_array[0]

# Columns:
# 0. Event name
# 1. Magnitude
# 2. Station name
# 3. Hypocentral distance (km)
# 4. xhigh
# 5. m1 slope
# 6. m2 slope
# 7. intersection point
# 8. sigma
# 9. transition time

In [ ]:
# Calculate earthquake source durations

# Get M0 from Mw, then use relation in Melgar and Hayes, 2017 to get duration

mags_for_relation = np.arange(6,7.25,0.05)
# print(mags_for_dur)

# Preferred params for source duration
apref_sd = -6.407
bpref_sd = 0.385

apref_rt = -5.323
bpref_rt = 0.293

source_durs = []
rise_times = []

for Mw in mags_for_relation:
    
    M0 = 10**(1.5*Mw + 9.05)
    # print(M0)
    
    log_sd = apref_sd + bpref_sd * np.log10(M0)
    source_dur = 10**log_sd
    source_durs.append(source_dur)
    
    log_rt = apref_rt + bpref_rt * np.log10(M0)
    rise_time = 10**log_rt
    rise_times.append(rise_time)
    

In [ ]:
plt.scatter(mcmc_array[:,3].astype(float), mcmc_array[:,9].astype(float)-10)
plt.xlabel('Hypocentral distance (km)')
plt.ylabel('Transition time (s)')
plt.show()

plt.scatter(mcmc_array[:,1].astype(float), mcmc_array[:,9].astype(float)-10)
plt.plot(mags_for_relation, source_durs, color = 'lime', label = 'Source duration scaling')
plt.plot(mags_for_relation, rise_times, color = 'red', label = 'Rise time scaling')
plt.xlabel('Earthquake magnitude')
plt.ylabel('Transition time (s)')
plt.semilogy()
plt.legend()
plt.show()

plt.scatter(mcmc_array[:,1].astype(float), mcmc_array[:,5].astype(float))
plt.xlabel('Earthquake magnitude')
plt.ylabel('Line 1 slope')
plt.show()

plt.scatter(mcmc_array[:,1].astype(float), mcmc_array[:,6].astype(float))
plt.xlabel('Earthquake magnitude')
plt.ylabel('Line 2 slope')
plt.show()

In [ ]:
# mcmc_array = np.column_stack((np.array(events_list), np.array(mags_list), np.array(stas_list), np.array(hypdist_list),
#                              np.array(xhighs_list), np.array(m1s_list), np.array(m2s_list), np.array(xinters_list),
#                              np.array(sigmas_list), np.array(trans_time_list)))

pd_mcmc = pd.DataFrame(mcmc_array, columns = ['eq_id', 'mag', 'sta', 'hypdist', 'xhigh', 'm1', 'm2', 'xinter', 'sigma', 'trans_time'])
pd_mcmc.to_csv(path_to_files + 'filtered_mcmc_arrays/mcmc_results_summary.csv')

In [31]:
# mcmc_results_class_by_eye = pd.read_csv(path_to_files + 'mcmc_results_summary.csv', usecols = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
# mcmc_results_class_by_eye

,eq_id,mag,sta,hypdist,xhigh,m1,m2,xinter,sigma,trans_time,behavior
0,2023-04-13_M6,6.0,B003,423,60,0.276567,0.012251,14.075615,0.141517,14.1,b
1,2023-04-13_M6,6.0,B004,398,60,0.523278,0.010967,12.877702,0.101206,12.9,g
2,2023-04-13_M6,6.0,B005,468,60,0.392836,0.009337,12.057298,0.075473,12.1,g
3,2023-04-13_M6,6.0,B009,456,60,0.428682,0.000518,12.471551,0.079256,12.5,g
4,2023-04-13_M6,6.0,B012,299,60,0.420404,0.013704,13.220529,0.112984,13.3,g
...,...,...,...,...,...,...,...,...,...,...,...
321,2010-01-10_M6.5,6.5,B066,499,35,0.228458,0.026041,15.608228,0.096790,15.7,g
322,2010-01-10_M6.5,6.5,B933,89,35,1.205989,0.046212,12.710480,0.220511,12.8,g
323,2010-01-10_M6.5,6.5,B934,123,35,0.258354,0.063122,12.957077,0.087686,13.0,b
324,2010-01-10_M6.5,6.5,B935,96,35,1.338175,0.021897,12.540953,0.210012,12.6,g


In [41]:
# good_results = mcmc_results_class_by_eye[mcmc_results_class_by_eye['behavior'] == 'g']
# bad_results = mcmc_results_class_by_eye[mcmc_results_class_by_eye['behavior'] == 'b']
# good_results

,eq_id,mag,sta,hypdist,xhigh,m1,m2,xinter,sigma,trans_time,behavior
1,2023-04-13_M6,6.0,B004,398,60,0.523278,0.010967,12.877702,0.101206,12.9,g
2,2023-04-13_M6,6.0,B005,468,60,0.392836,0.009337,12.057298,0.075473,12.1,g
3,2023-04-13_M6,6.0,B009,456,60,0.428682,0.000518,12.471551,0.079256,12.5,g
4,2023-04-13_M6,6.0,B012,299,60,0.420404,0.013704,13.220529,0.112984,13.3,g
5,2023-04-13_M6,6.0,B927,349,60,0.644048,0.006067,12.064490,0.108054,12.1,g
...,...,...,...,...,...,...,...,...,...,...,...
319,2010-01-10_M6.5,6.5,B054,377,35,0.474619,0.001973,13.653904,0.078392,13.7,g
320,2010-01-10_M6.5,6.5,B057,344,35,0.379983,0.023154,14.384422,0.117667,14.4,g
321,2010-01-10_M6.5,6.5,B066,499,35,0.228458,0.026041,15.608228,0.096790,15.7,g
322,2010-01-10_M6.5,6.5,B933,89,35,1.205989,0.046212,12.710480,0.220511,12.8,g


In [51]:
# plt.figure(dpi = 400)
# plt.scatter(bad_results.hypdist.values, bad_results.xinter.values, marker = 'x', color = '#E9072B', label = '"Poorly-behaved" waveforms')
# plt.scatter(good_results.hypdist.values, good_results.xinter.values, marker = 'o', color = '#355464', alpha = 0.6, label = '"Well-behaved" waveforms')
# plt.xlabel('Hypocentral distance (km)')
# plt.ylabel('Transition time (s)')
# plt.legend()
# # plt.show()
# plt.savefig(path_to_files + 'figures/filtered_mcmc_plots/summary_hypdist_vs_transtime.png', format = 'PNG')
# plt.close()

# plt.figure(dpi = 400)
# plt.scatter(bad_results.mag.values, bad_results.xinter.values, marker = 'x', color = '#E9072B', label = '"Poorly-behaved" waveforms')
# plt.scatter(good_results.mag.values, good_results.xinter.values, marker = 'o', color = '#355464', alpha = 0.6, label = '"Well-behaved" waveforms')
# plt.xlabel('Earthquake magnitude')
# plt.ylabel('Transition time (s)')
# plt.legend()
# # plt.show()
# plt.savefig(path_to_files + 'figures/filtered_mcmc_plots/summary_mag_vs_transtime.png', format = 'PNG')
# plt.close()

# plt.figure(dpi = 400)
# plt.scatter(bad_results.mag.values, bad_results.m1.values, marker = 'x', color = '#E9072B', label = '"Poorly-behaved" waveforms')
# plt.scatter(good_results.mag.values, good_results.m1.values, marker = 'o', color = '#355464', alpha = 0.6, label = '"Well-behaved" waveforms')
# plt.xlabel('Earthquake magnitude')
# plt.ylabel('Line 1 slope')
# plt.legend()
# # plt.show()
# plt.savefig(path_to_files + 'figures/filtered_mcmc_plots/summary_mag_vs_m1.png', format = 'PNG')
# plt.close()

# plt.figure(dpi = 400)
# plt.scatter(bad_results.mag.values, bad_results.m2.values, marker = 'x', color = '#E9072B', label = '"Poorly-behaved" waveforms')
# plt.scatter(good_results.mag.values, good_results.m2.values, marker = 'o', color = '#355464', alpha = 0.6, label = '"Well-behaved" waveforms')
# plt.xlabel('Earthquake magnitude')
# plt.ylabel('Line 2 slope')
# plt.legend()
# # plt.show()
# plt.savefig(path_to_files + 'figures/filtered_mcmc_plots/summary_mag_vs_m2.png', format = 'PNG')
# plt.close()

In [93]:
summary = az.summary(mcmc)

In [94]:
summary

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
m1,1.608,0.390,1.130,2.124,0.274,0.231,2.0,11.0,2.84
m2,0.013,0.003,0.011,0.019,0.002,0.001,3.0,21.0,2.24
xinter,11.131,0.335,10.699,11.574,0.235,0.198,2.0,10.0,2.69
sigma,0.106,0.037,0.067,0.177,0.024,0.020,2.0,13.0,2.32


In [77]:
summary['mean']

m1         0.302
m2         0.039
xinter    12.971
sigma      0.163
Name: mean, dtype: float64

In [82]:
m1 = summary['mean'].iloc[0]
m2 = summary['mean'].iloc[1]
xinter = summary['mean'].iloc[2]
sigma = summary['mean'].iloc[3]

In [85]:
print(sigma)

0.163


In [87]:
# Done, now is post-processing to get the data out of the sampler

summary = az.summary(mcmc)

# Unwrap coeficients
m1 = summary['mean'].iloc[0]
m2 = summary['mean'].iloc[1]
xinter = summary['mean'].iloc[2]
sigma = summary['mean'].iloc[3]
b1 = y0 - m1*x0
b2 = m1*xinter + b1 - m2*xinter

In [88]:
# Make plot to check stuff
xpredicted = np.arange(xobserved.min(), xobserved.max()+0.1, 0.1)
ypredicted = np.ones(len(xpredicted))
ypredicted = m1*xpredicted + b1
i = np.where(xpredicted > xinter)[0]
ypredicted[i] = m2*xpredicted[i] + b2

In [90]:
posterior = az.extract(mcmc)

In [95]:
posterior

<xarray.Dataset> Size: 224kB
Dimensions:  (sample: 4000)
Coordinates:
  * sample   (sample) object 32kB MultiIndex
  * chain    (sample) int64 32kB 0 0 0 0 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1 1 1 1 1
  * draw     (sample) int64 32kB 0 1 2 3 4 5 6 ... 1994 1995 1996 1997 1998 1999
Data variables:
    m1       (sample) float64 32kB 0.2794 0.2794 0.2794 ... 0.3114 0.3114 0.3114
    m2       (sample) float64 32kB 0.0466 0.0466 0.0466 ... 0.04077 0.04077
    xinter   (sample) float64 32kB 12.98 12.98 12.99 12.99 ... 12.95 12.94 12.94
    sigma    (sample) float64 32kB 0.1522 0.1522 0.1522 ... 0.1575 0.1575 0.1575
Attributes:
    created_at:                 2024-03-05T10:45:23.481163
    arviz_version:              0.17.0
    inference_library:          pymc
    inference_library_version:  5.10.4
    sampling_time:              8.208245038986206
    tuning_steps:               1000

In [99]:
len(posterior.m1.values)

4000